In [1]:
!ls

__pycache__       main.py           static            vertexSearches.py
debugger.ipynb    publishing.py     templates         video2BQ.py
Dockerfile        requirements.txt  venv              video2images.py


In [5]:
!python3.12 -m pip install --upgrade pip
!pip3 install flask
!pip3 install google-cloud-bigquery

In [6]:
#from flask import Flask, render_template, request, jsonify, send_from_directory
from video2BQ import generate, grounded_upc
from publishing import sendPS
import json
import threading
import os
# Add GCS and BQ imports
from google.cloud import storage
from google.cloud import bigquery

# GCS and BQ Configuration (Consider moving to config or env vars)
GCS_BUCKET_NAME = "public-bdoohan-bucket"
BQ_PROJECT_ID = "kiran-cailin"
BQ_DATASET_ID = "spielbergWebapp"
BQ_TABLE_ID = "results_per_store"
BQ_FULL_TABLE_ID = f"{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}"


# Initialize clients (consider lazy initialization or application context)
storage_client = storage.Client()
bq_client = bigquery.Client(project=BQ_PROJECT_ID)

#from flask import Flask
#app = Flask(__name__)

#app = Flask(__name__, static_folder='static')

# Global variable to store processing status
processing_status = {
    "is_processing": False,
    "result": None,
    "current_file": None,
    "bq_row": None,
    "summary": None # Added for grounded_upc results
}

def get_filename_from_uri(uri):
    # Handle potential gs:// prefix
    if uri.startswith("gs://"):
        # Find the first slash after gs://
        slash_index = uri.find('/', 5)
        if slash_index != -1:
            return os.path.basename(uri[slash_index+1:]) # Get part after bucket name
    return os.path.basename(uri) # Fallback for other formats or simple names


def process_video(video_uri):
    global processing_status
    processing_status["is_processing"] = True
    processing_status["result"] = None # Clear previous results
    processing_status["summary"] = None # Clear previous results
    processing_status["bq_row"] = None # Clear previous BQ row
    processing_status["current_file"] = get_filename_from_uri(video_uri)
    try:
        # Modify the video_gcs_uri in the generate function
        # The 'generate' function now returns the raw Gemini JSON output
        gemini_raw_result = generate(video_uri)
        processing_status["result"] = {"status": "success", "message": gemini_raw_result} # Store raw Gemini result
        
        # Pass the raw Gemini result (string) to grounded_upc
        summary_results = grounded_upc(gemini_raw_result)
        processing_status["summary"] = {"status": "success", "message": summary_results}

        # Extract the BigQuery row information from the result
        # Note: 'generate' currently doesn't return bq_row directly.
        # If BigQuery row info is needed, it must be returned by 'generate'
        # or fetched separately. For now, it might be None if 'generate'
        # only returns the Gemini text.
        # This part of the code expects 'generate' to return a dict with 'bq_row'.
        # Given the updated generate, this part might need adjustment
        # if you want to store the BQ row specifically here.
        # For demonstration, let's keep it as is, assuming generate might evolve.
        # Or you could fetch it from BQ preview if needed.
        # For now, it will likely be None if generate just returns the string.
        if isinstance(gemini_raw_result, dict) and "bq_row" in gemini_raw_result:
            processing_status["bq_row"] = gemini_raw_result["bq_row"]
        else:
            processing_status["bq_row"] = {"message": "BQ row details not directly returned by generate function."}


    except Exception as e:
        processing_status["result"] = {"status": "error", "message": f"Processing error: {str(e)}"}
        processing_status["summary"] = ["Error during grounding: " + str(e)]
        processing_status["bq_row"] = None
    finally:
        processing_status["is_processing"] = False

def list_videos():
    """Lists .mp4 and .mov files from the specified GCS bucket."""
    try:
        blobs = storage_client.list_blobs(GCS_BUCKET_NAME)
        video_files = []
        for blob in blobs:
            if blob.name.lower().endswith(('.mp4', '.mov')):
                 # Return full gs:// path
                video_files.append(f"gs://{GCS_BUCKET_NAME}/{blob.name}")
        return jsonify({"status": "success", "videos": video_files})
    except Exception as e:
        print(f"Error listing GCS bucket: {e}")
        return jsonify({"status": "error", "message": f"Error listing GCS bucket: {str(e)}"}), 500

def bq_preview():
    """Fetches the first 3 rows from the BigQuery results table."""
    try:
        # Corrected table ID usage
        query = f"""
            SELECT *
            FROM `{BQ_FULL_TABLE_ID}`
            ORDER BY time DESC
            LIMIT 3
        """
        query_job = bq_client.query(query) # API request
        results = query_job.result() # Waits for job to complete

        rows = [dict(row) for row in results]

        # Convert datetime objects to string for JSON serialization
        for row in rows:
             if 'timestamp' in row and hasattr(row['timestamp'], 'isoformat'):
                 row['timestamp'] = row['timestamp'].isoformat()

        return jsonify({"status": "success", "rows": rows})
    except Exception as e:
        print(f"Error querying BigQuery: {e}")
        return jsonify({"status": "error", "message": f"Error querying BigQuery: {str(e)}"}), 500


def process():
    if processing_status["is_processing"]:
        return jsonify({"status": "error", "message": "Already processing a video"})

    video_uri = request.json.get('video_uri')
    if not video_uri:
        return jsonify({"status": "error", "message": "No video URI provided"})

    # Start processing in a background thread
    thread = threading.Thread(target=process_video, args=(video_uri,))
    thread.start()

    return jsonify({"status": "success", "message": "Processing started"})

def status():
    # Make sure result and summary are serializable
    serializable_status = processing_status.copy()
    if isinstance(serializable_status.get("result"), Exception):
         serializable_status["result"] = {"status": "error", "message": str(serializable_status["result"])}
    
    # Ensure summary is a list of strings or None
    if serializable_status.get("summary") is not None and not isinstance(serializable_status["summary"], list):
        serializable_status["summary"] = [str(serializable_status["summary"])]
    elif serializable_status.get("summary") is not None and isinstance(serializable_status["summary"], list):
        serializable_status["summary"] = [str(item) for item in serializable_status["summary"]]


    return jsonify(serializable_status)

ModuleNotFoundError: No module named 'google'